In [2]:
using Revise

In [3]:
using ReverseDiff
using ReverseDiff: @forward
using ForwardDiff
using DiffResults
using CoordinateTransformations

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/ReverseDiff.ji for module ReverseDiff.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/Rotations.ji for module Rotations.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/CoordinateTransformations.ji for module CoordinateTransformations.


In [11]:
import Nets2
reload("Nets2")

In [12]:
c = Nets2.Chain(AffineMap(0.9 .* ones(1, 1), 0.1 .+ zeros(1)))

Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}}}}((AffineMap([0.9], [0.1]),))

In [13]:
c([0.5])

1-element Array{Float64,1}:
 0.55

In [14]:
c = Nets2.Chain(
    AffineMap(0.9 .* ones(1, 1), 0.1 .+ zeros(1)),
    Nets2.Affine(fill(1.0, 1, 1), fill(0.5, 1)),
    Nets2.activation(Nets2.elu)
    )

Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##3#4{Nets2.#elu}}}((AffineMap([0.9], [0.1]), Nets2.Affine, Nets2.#3))

In [15]:
c([0.5])

1-element Array{Float64,1}:
 1.05

In [16]:
ct = Nets2.TangentPropagation(c)

Nets2.TangentPropagation{Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##3#4{Nets2.#elu}}}}(Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##3#4{Nets2.#elu}}}((AffineMap([0.9], [0.1]), Nets2.Affine, Nets2.#3)))

In [17]:
ct([0.5])

([1.05], [0.9])

In [18]:
loss(yJ) = sum(yJ[1] + yJ[2])

loss (generic function with 1 method)

In [20]:
params = Nets2.params(ct)
ReverseDiff.gradient((p...) -> loss(Nets2.with_params(ct, p...)([1.0])), params)

([1.9], [1.0])

In [21]:
p = copy.(params)
p[1] .+= 1e-5
(loss(Nets2.with_params(ct, p...)([1.0])) - loss(Nets2.with_params(ct, params...)([1.0]))) / 1e-5

1.9000000000435333

In [22]:
Nets2.params(c)

([1.0], [0.5])

In [23]:
c([1.0])

1-element Array{Float64,1}:
 1.5

In [24]:
params = (fill(1.0, 1, 1), [0.25])

([1.0], [0.25])

In [25]:
c2 = Nets2.with_params(c, params...)

Nets2.Chain{Tuple{CoordinateTransformations.AffineMap{Array{Float64,2},Array{Float64,1}},Nets2.Affine{Array{Float64,2},Array{Float64,1}},Nets2.##3#4{Nets2.#elu}}}((AffineMap([0.9], [0.1]), Nets2.Affine, Nets2.#3))

In [26]:
ReverseDiff.gradient((p...) -> Nets2.with_params(c, p...)([1.0])[1], params)

([1.0], [1.0])

In [27]:
f = (params, x) -> x .+ params
ReverseDiff.jacobian(p -> sum((f(p, [1.0]), ForwardDiff.jacobian(x -> f(p, x), [1.0]))), [1.0])

1×1 Array{Float64,2}:
 1.0

In [28]:
c = Nets2.Chain(
#     AffineMap(0.9 .* ones(1, 1), 0.1 .+ zeros(1)),
    Nets2.Affine(fill(1.0, 1, 1), fill(0.5, 1)),
#     Nets2.activation(Nets2.elu)
    )

Nets2.Chain{Tuple{Nets2.Affine{Array{Float64,2},Array{Float64,1}}}}((Nets2.Affine,))

In [30]:
ReverseDiff.gradient((p...) -> sum(Nets2.with_params(ct, p...)([1.0]))[1], params)

([1.9], [1.0])

In [34]:
x = [2.0]
yJ = (3.0, fill(1.5, 1, 1))
net = Nets2.TangentPropagation(c)
Nets2.initialize!(randn, Nets2.params(net))

loss(params) = sum(abs2, Nets2.with_params(net, params...)(x) .- yJ)

@show loss(Nets2.params(net))

params = Nets2.params(net)
for i in 1:100
    ∇ = ReverseDiff.gradient(loss, params)
    params = params .- 0.01 .* ∇
end
    
@show loss(Nets2.params(net))

loss(Nets2.params(net)) = [20.0111]


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] abs2(::Array{Float64,1}) at ./deprecated.jl:57
 [3] mapfoldl(::Base.#abs2, ::Function, ::Tuple{Array{Float64,1},Array{Float64,2}}) at ./reduce.jl:72
 [4] sum(::Function, ::Tuple{Array{Float64,1},Array{Float64,2}}) at ./reduce.jl:347
 [5] loss(::Tuple{Array{Float64,2},Array{Float64,1}}) at ./In[34]:6
 [6] include_string(::String, ::String) at ./loading.jl:522
 [7] include_string(::Module, ::String, ::String) at /home/rdeits/locomotion/explorations/learning-mpc/packages/v0.6/Compat/src/Compat.jl:174
 [8] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/rdeits/locomotion/explorations/learning-mpc/packages/v0.6/IJulia/src/execute_request.jl:154
 [9] (::Compat.#inner#16{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/rdeits/locomotion/explorations/learning-mpc/packages/v0.6/Compat/src/Compat.jl:496
 [10] eventloop(::ZMQ.Socket) at /home/rdeits/locomotion/explorations/learning-mpc/packa

LoadError: [91mMethodError: no method matching loss(::ReverseDiff.TrackedArray{Float64,Float64,2,Array{Float64,2},Array{Float64,2}}, ::ReverseDiff.TrackedArray{Float64,Float64,1,Array{Float64,1},Array{Float64,1}})[0m
Closest candidates are:
  loss(::Any) at In[34]:6[39m